Toutes les salles de la carte ont un identifiant unique dans le catalogue. Connaître l'école du tableau ne renseigne pas vraiment d'avantage sur sa salle.

Pour utiliser le processeur, il faut indiquer le nom de la salle en création en haut, et de la salle connectée à la porte en création juste en dessous. Pour créer les murs, cliquer sur les coins. Pour enclencher ou déclencher le mode "porte", appuyer sur la flèche droite.

In [1]:
import pandas as pd

import PIL.Image

from tkinter import *

import ast

In [11]:
path = "Louvre_map/Louvre_map_c.png"

im = PIL.Image.open(path)
width, height = im.size
#im.thumbnail((int(0.5*width),int(0.5*height)))
#width, height = im.size

In [12]:
# A faire : ajouter une boite de sélection pour le choix de la salle et de toutes les portes (mettre plusieurs boites)
# Ajouter la possibilité d'annuler la création de salle en cours
# Ou alors contrôler dans le nettoyage des données que les salles ont le bon nom et simplement placer une liste
# indicative ici ?

#On notera qu'à cause du traitement de l'OCR, le numéro des salles des meubles XVIII est perdu !
valid_rooms = []
with open("Rooms.txt","r",encoding = "utf-8") as f:
    for line in f:
        valid_rooms.append(line.replace("é","e").split("/")[0].replace("\n",""))

In [13]:
corner = 0
door = False
ongoing_door = False
door_first_coord = None
door_counter = 0

x = 0
y = 0

old_x = 0
old_y = 0

anchor_x = 0
anchor_y = 0

rooms = []
room = {}

r = 3

window = Tk()

canvas = Canvas(window,width=width,height=height)

louvre = PhotoImage(master = canvas,file=path)

def init_image():
    canvas.create_image(0, 0, image=louvre, anchor=NW)

init_image()

roomBox=Text(window, height=1, width=50)
roomBox.pack()

doorBox=Text(window, height=1, width=100)
doorBox.pack()

roomsLabel = Label(window, text = " - ".join(valid_rooms),wraplength = width-200)
roomsLabel.pack()

def draw_circle(X,Y,color):
    canvas.create_oval(X-r,Y-r,X+r,Y+r,fill = color)
    
def create_door(X,Y):
    global ongoing_door
    global room
    global door_first_coord
    global anchor_x
    global anchor_y
    global door_counter
    print("Creating_door")
    if ongoing_door:
        doorText = doorBox.get("1.0","end-1c").split(", ")[door_counter]
        door_counter += 1
        room["doors"].append((doorText,door_first_coord,(X,Y)))
        ongoing_door = False
    else:
        door_first_coord = (X,Y)
        ongoing_door = True
    draw_circle(anchor_x + X,anchor_y + Y,"blue")

def callback(event):
    print("clicked at", event.x, event.y)
    global room
    global rooms
    global corner
    global x
    global y
    global old_x
    global old_y
    global anchor_x
    global anchor_y
    global door_counter
    if corner == 0:
        if door:
            x = 0
            y = min(max(event.y - anchor_y,0),old_y)
            create_door(x,y)
        else:
            print("Creating new room")
            if room:
                rooms.append(room)
            anchor_x = event.x
            anchor_y = event.y
            x = 0
            y = 0
            old_x = 0
            old_y = 0
            room = {"name":roomBox.get("1.0","end-1c"),"doors":[],"NO":(x,y)}
            init_image()
            draw_circle(x+anchor_x,y+anchor_y,"red")
            corner += 1
            door_counter = 0
    
    elif corner == 1:
        x = max(event.x-anchor_x,old_x)
        y = old_y
        if door:
            create_door(max(x,0),y)
        else:
            room["NE"] = (x,y)
            draw_circle(x+anchor_x,y+anchor_y,"red")
            corner += 1
    
    elif corner == 2:
        x = old_x
        y = max(event.y-anchor_y,old_y)
        if door:
            create_door(x,max(y,0))
        else:
            room["SE"] = (x,y)
            draw_circle(x+anchor_x,y+anchor_y,"red")
            corner += 1
    
    else:
        x = max(min(event.x-anchor_x,old_x),0)
        y = old_y
        if door:
            create_door(x,y)
        else:
            room["SO"] = (0,y)
            draw_circle(anchor_x,y+anchor_y,"red")
            corner += 1
        
    old_x = x
    old_y = y
    corner = corner%4

                    
def door_change(event):
    global door
    door = not door 
    
canvas.bind("<Button-1>", callback)
window.bind("<Right>",door_change)
canvas.pack()

window.mainloop()

In [22]:
print(rooms)

rooms_df = pd.DataFrame(rooms)

[{'name': 'Ire S Meubles XVIIIe', 'doors': [('IIe S Meubles XVIIIe', (9, 0), (21, 0)), ('IIe S Meubles XVIIIe', (48, 0), (65, 0)), ('S Meubles XVIIe', (65, 75), (52, 75)), ('S Meubles XVIIe', (22, 75), (7, 75))], 'NO': (0, 0), 'NE': (65, 0), 'SE': (65, 75), 'SO': (0, 75)}, {'name': 'IIe S Meubles XVIIIe', 'doors': [('IIIe S Meubles XVIIIe', (5, 0), (22, 0)), ('IIIe S Meubles XVIIIe', (49, 0), (68, 0)), ('Ie S Meubles XVIIIe', (68, 37), (54, 37)), ('Ie S Meubles XVIIIe', (20, 37), (6, 37))], 'NO': (0, 0), 'NE': (68, 0), 'SE': (68, 37), 'SO': (0, 37)}, {'name': 'IIIe S Meubles XVIIIe', 'doors': [('VIe S Meubles XVIIIe', (0, 0), (20, 0)), ('IIe S Meubles XVIIIe', (66, 38), (49, 38)), ('IIe S Meubles XVIIIe', (19, 38), (6, 38))], 'NO': (0, 0), 'NE': (66, 0), 'SE': (66, 38), 'SO': (0, 38)}, {'name': 'IVe S Meubles XVIIIe', 'doors': [('S Barye', (84, 7), (84, 17)), ('E F XIXe S 1er etage I', (84, 37), (84, 52)), ('IIIe S Meubles XVIIIe', (41, 66), (19, 66))], 'NO': (0, 0), 'NE': (84, 0), 'SE

In [23]:
import os

if not os.path.exists('data/rooms.csv'):
    rooms_df.to_csv("data/rooms.csv")
else:
    with open('data/rooms.csv', 'a') as f:
        rooms_df.to_csv(f, header=False)

#rooms_df.to_excel("temp/rooms.xlsx")

## File formatting

In [24]:
#Real scale is 1/4, but the scaling is done in Unity
def create_wall(row,target,corner_1,corner_2, scale = 1):
        
    dic = {}
    
    dic["card"] = target
    p00, p01 = row[corner_1].strip(")(").split(", ")
    p10, p11 = row[corner_2].strip(")(").split(", ")
       
    dic["p00"], dic["p01"], dic["p10"], dic["p11"] = scale*int(p00), scale*int(p01), scale*int(p10), scale*int(p11)
    
    return dic

def create_doors(text, scale = 1):
    l = ast.literal_eval(text)
    
    result = []
    
    for door in l:
        
        dic = {}
        
        dic["name"] = door[0].strip("'")
        p00, p01 = door[1][0], door[1][1]
        p10, p11 = door[2][0], door[2][1]
        
        dic["p00"], dic["p01"], dic["p10"], dic["p11"] = scale*int(p00), scale*int(p01), scale*int(p10), scale*int(p11)
        
        result.append(dic)
    
    return result
   

In [25]:
full_rooms_df = pd.read_csv("data/rooms.csv", encoding = "utf-8")
full_rooms_df

,Unnamed: 0,name,doors,NO,NE,SE,SO
0,0,Coll Schlichting,"[('Coll Chauchard galerie', (137, 11), (137, 1...","(0, 0)","(137, 0)","(137, 30)","(0, 30)"
1,1,Coll Chauchard galerie,"[('XXVIII', (220, 0), (236, 0)), ('XVIII', (23...","(0, 0)","(236, 0)","(236, 23)","(0, 23)"
2,2,XXVIII,"[('XXIX', (37, 0), (37, 7)), ('Coll Chauchard ...","(0, 0)","(37, 0)","(37, 30)","(0, 30)"
3,3,Coll Chauchard IV,"[('Coll Chauchard III', (58, 0), (58, 10)), ('...","(0, 0)","(58, 0)","(58, 48)","(0, 48)"
4,4,Coll Chauchard III,"[('Coll Chauchard galerie', (8, 0), (17, 0)), ...","(0, 0)","(33, 0)","(33, 57)","(0, 57)"
...,...,...,...,...,...,...,...
71,5,Esc T T,"[('E F XIXe S 2e etage I', (50, 34), (42, 34))...","(0, 0)","(50, 0)","(50, 34)","(0, 34)"
72,0,Ire S Meubles XVIIIe,"[('IIe S Meubles XVIIIe', (9, 0), (21, 0)), ('...","(0, 0)","(65, 0)","(65, 75)","(0, 75)"
73,1,IIe S Meubles XVIIIe,"[('IIIe S Meubles XVIIIe', (5, 0), (22, 0)), (...","(0, 0)","(68, 0)","(68, 37)","(0, 37)"
74,2,IIIe S Meubles XVIIIe,"[('IVe S Meubles XVIIIe', (0, 0), (20, 0)), ('...","(0, 0)","(66, 0)","(66, 38)","(0, 38)"


In [26]:
for index, row in full_rooms_df.iterrows():
    
    walls = [create_wall(row,"n","NO","NE"),create_wall(row,"e","NE","SE"),\
             create_wall(row,"s","SE","SO"),create_wall(row,"o","SO","NO")]    
    walls_df = pd.DataFrame(walls)
    
    doors_df = pd.DataFrame(create_doors(row.doors))
    
    walls_df.to_json("data/walls_json/{}.json".format(row["name"]), orient="records")
    doors_df.to_json("data/doors_json/{}.json".format(row["name"]), orient="records")

# Tests Divers

In [7]:
if {}:
    print("Plop")

In [8]:
if {"a":"b"}:
    print("Plop")

Plop


In [3]:
rooms_df.doors

NameError: name 'rooms_df' is not defined

In [72]:
list(ast.literal_eval("[('Esc Daru', (257, 10), (257, 29)), ('XV', (0, 47), (0, 29)), ('XV', (0, 14), (0, 5))]"))[0][0]

'Esc Daru'